In [6]:
## this code should not be neccessary to run, it downloads the data from kaggle

#%pip install -U -q kaggle
#%mkdir -p ~/.kaggle
#!echo '{"username":"xxxx","key":"xxxxxxxxx"}' > ~/.kaggle/kaggle.json  # change to you own kaggle profile API token
#!chmod 600 ~/.kaggle/kaggle.json
#!kaggle datasets download -d Cornell-University/arxiv -p "../data"

Note: you may need to restart the kernel to use updated packages.
100%|█████████████████████████████████████▉| 1.21G/1.21G [05:32<00:00, 4.64MB/s]
100%|██████████████████████████████████████| 1.21G/1.21G [05:32<00:00, 3.91MB/s]


In [7]:
## you don't have to run this code too
#!unzip -n '../data/arxiv.zip' -d '../data'

Archive:  ../data/arxiv.zip
  inflating: ../data/arxiv-metadata-oai-snapshot.json  


In [12]:
import json

## read in ALL the data, it may take a while
#data = [json.loads(line)
#        for line in open('data/arxiv-metadata-oai-snapshot.json', 'r', encoding='utf-8')]


## use the following code to read in limited number of data
def read_in_data(output_folder):
    data = []
    i = 0
    for line in open('data/arxiv-metadata-oai-snapshot.json', 'r', encoding='utf-8', errors="replace"):
        data.append(json.loads(line))
        i = i + 1
        if i == 10:    # you can change the limit of reading in data objects by changing the number in if- condition
            break
    
    ## if everything went well this line should print out the first element of the list
    #print(json.dumps(data[0], indent=2))   
    # data cleaning example: "droping" data with one word titles
    data = [obj for obj in data if len(obj['title'].split(" ")) > 1]
    modified_data = []

    ## line by line data modification
    for line in data:
        modified_data.append(line)
        print(line.get("authors_parsed"))

    #with open(f'{output_folder}/cleaned_data.json', 'w') as json_file:
    #    json.dump(modified_data, json_file, indent=2)
    #return data     
read_in_data("a")

[['Balázs', 'C.', ''], ['Berger', 'E. L.', ''], ['Nadolsky', 'P. M.', ''], ['Yuan', 'C. -P.', '']]
[['Streinu', 'Ileana', ''], ['Theran', 'Louis', '']]
[['Pan', 'Hongjun', '']]
[['Callan', 'David', '']]
[['Abu-Shammala', 'Wael', ''], ['Torchinsky', 'Alberto', '']]
[['Pong', 'Y. H.', ''], ['Law', 'C. K.', '']]
[['Corichi', 'Alejandro', ''], ['Vukasinac', 'Tatjana', ''], ['Zapata', 'Jose A.', '']]
[['Swift', 'Damian C.', '']]
[['Harvey', 'Paul', ''], ['Merin', 'Bruno', ''], ['Huard', 'Tracy L.', ''], ['Rebull', 'Luisa M.', ''], ['Chapman', 'Nicholas', ''], ['Evans', 'Neal J.', 'II'], ['Myers', 'Philip C.', '']]
[['Ovchinnikov', 'Sergei', '']]


In [36]:
print([x['id'] for x in data if len(x['versions']) > 3])

def clean_data(output_folder):

    with open(f'{output_folder}/cleaned_data.json', 'r') as f:
        data = json.load(f)
    # data cleaning example: "droping" data with one word titles
    data = [obj for obj in data if len(obj['title'].split(" ")) > 1]
    modified_data = []

    ## line by line data modification
    for line in data:
        modified_data.append(line)


['0704.0027', '0704.0041', '0704.0044', '0704.0063', '0704.0064', '0704.0080', '0704.0098']


In [44]:
#%pip install requests
import requests

def getArticleResource(output_folder):
    
    with open(f'{output_folder}/cleaned_data.json', 'r') as f:
        data = json.load(f)
    #url = "https://serpapi.com/search"
    url = "https://dblp.org/search/publ/api"

    ##params = {
    ##"engine": "google_scholar",
    ##"q": title,
    ##"api_key": "95e72264387e5901953190dbc9608e0a4bc625ae3e2d4b3b3dca1c3995d628d8"
    ##}
    for obj in data:
        #print(obj.get("title"))
        #print(getArticleResource(obj.get("title")))

        params = {
        "q": obj.get("title"),
        "format": "json",
        "h": 20
        }

        response = requests.get(url, params=params)
        if response.status_code == 200:
            results = response.json().get("result").get("hits")
            if int(results.get("@total")) == 0:
                #return None
                break
            for hit in results.get("hit"):
                #print(hit.get("info").get("url"))
                if hit.get("info").get("title") == obj.get("title"):
                    #return hit.get("info").get("url")
                    obj['url'] = hit.get("info").get("url")
            #organic_results = results.get("organic_results")
            #return organic_results
            #return None
    with open(f'{output_folder}/augmented_data.json', 'w') as json_file:
        json.dump(data, json_file, indent=2)

#print(getArticleResource("Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies"))
#print(getArticleResource("test"))



None


In [45]:
#for obj in modified_data:
#    #print(obj.get("title"))
#    print(getArticleResource(obj.get("title")))

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [ ]:
from airflow import DAG 
from airflow.operators.python_operator import PythonOperator, BranchPythonOperator
from airflow.operators.bash_operator import BashOperator 
from airflow.operators.dummy_operator import DummyOperator
from airflow.providers.postgres.operators.postgres import PostgresOperator
from airflow.contrib.sensors.file_sensor import FileSensor
from datetime import datetime, timedelta


DEFAULT_ARGS = {
    'owner': 'Tartu',
    'depends_on_past': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5)
}

DATA_FOLDER = '/tmp/data'

dag = DAG(
    dag_id='ingestion', # name of dag
    schedule_interval='01 0 * * *', # execute every minute
    start_date=datetime(2023,9,14,9,15,0),
    catchup=False, # in case execution has been paused, should it execute everything in between
    template_searchpath=DATA_FOLDER, # the PostgresOperator will look for files in this folder
    default_args=DEFAULT_ARGS, # args assigned to all operators
)

file_sensor = FileSensor(
    task_id='file_sensor',
    filepath='/data/staging',
    fs_conn_id='fs_default',
    poke_interval=5,
    dag=dag
)

first_task = PythonOperator(
    task_id='get_data',
    dag=dag,
    trigger_rule='none_failed',
    python_callable=read_in_data,
    op_kwargs={
        'output_folder': DATA_FOLDER
    },
)

#file_sensor >> first_task

second_task = PythonOperator(
    task_id='augment_data',
    dag=dag,
    trigger_rule='none_failed',
    python_callable=getArticleResource,
    op_kwargs={
        'output_folder': DATA_FOLDER
    },
)

first_task >> second_task

def insert_categories(output_folder):
    with open(f'{output_folder}/augmented_data.json', 'r') as f:
        data = json.load(f)
    sql_file = ''
    unique_cat = set()

    for row in data:
        if row.get("categories") not in unique_cat:
            unique_cat.add(row.get("categories"))
    
    #article_id = 1
    cat_id = 1
    #author_id = 1
    #article_author_id = 1
    for cat in unique_cat:
        sql_file = sql_file + f'INSERT INTO category (ID, name) VALUES (\'{cat_id}\', \'{cat}\');\n'
        cat_id += 1
    
    with open(f'{output_folder}/insert_categories.sql', 'w') as f:
        f.write(sql_file)


def insert_authors(output_folder):
    with open(f'{output_folder}/augmented_data.json', 'r') as f:
        data = json.load(f)
    sql_file = ''
    unique_authors = set()

    for row in data:
        if len(row.get("authors").split(",")) == 0:
            if author.strip() not in unique_authors:
                unique_authors.add(author.strip())
        else:
            authors = row.get("authors").split(",")
            for author in authors:
                if author.strip() not in unique_authors:
                    unique_authors.add(author.strip())
    
    #article_id = 1
    author_id = 1
    #article_author_id = 1
    for author in unique_authors:
        if len(author.split(" ")) == 0:
            names_separated = ["", author]
        else:
            names_separated = author.rsplit(" ", 1)
        sql_file = sql_file + f'INSERT INTO author (ID, first_name, last_name) VALUES (\'{author_id}\', \'{names_separated[0]}\', \'{names_separated[1]}\');\n'
        author_id += 1
    
    with open(f'{output_folder}/insert_authors.sql', 'w') as f:
        f.write(sql_file)


def insert_articles(output_folder):
    with open(f'{output_folder}/augmented_data.json', 'r') as f:
        data = json.load(f)
    sql_file = ''
    
    for row in data:
        sql_file = sql_file + f'INSERT INTO article (ID, title, doi, update_date, journal_ref, category_id) 
        VALUES (\'{row.get("id")}\', \'{row.get("title")}\',\'{row.get("doi")}\',\'{row.get("update_date")}\',\'{row.get("journal-ref")}\',
        (SELECT ID FROM category WHERE name = \'{row.get("categories")}\'));\n'
    
    with open(f'{output_folder}/insert_articles.sql', 'w') as f:
        f.write(sql_file)


def insert_articles_authors(output_folder):
    with open(f'{output_folder}/augmented_data.json', 'r') as f:
        data = json.load(f)
    sql_file = ''
    
    article_author_id = 1
    for row in data:
        if len(row.get("authors").split(",")) == 0:
            authors = [row.get("authors").split(",")]
        else:
            authors = row.get("authors").split(",")
        for author in authors:
            if len(author.split(" ")) == 0:
                names_separated = ["", author]
            else:
                names_separated = author.rsplit(" ", 1)
            sql_file = sql_file + f'INSERT INTO article_author (ID, article_id, author_id) 
            VALUES (\'{article_author_id}\', \'{row.get("id")}\', (SELECT ID FROM author WHERE first_name = \'{names_separated[0]}\' AND last_name=\'{names_separated[1]}\'));\n'
                
    with open(f'{output_folder}/insert_article_author.sql', 'w') as f:
        f.write(sql_file)


create_tables = PostgresOperator(
    task_id="create_tables",
    postgres_conn_id="postgres_default",
    sql="sql/create_tables.sql",
    dag=dag
)

second_task >> create_tables

third_task = PythonOperator(
    task_id='insert_categories',
    dag=dag,
    trigger_rule='none_failed',
    python_callable=insert_categories,
    op_kwargs={
        'output_folder': DATA_FOLDER,
    },
)

create_tables >> third_task

fourth_task = PostgresOperator(
    task_id='insert_categories_to_db',
    dag=dag,
    postgres_conn_id='airflow_pg',
    sql='insert_categories.sql',
    trigger_rule='none_failed',
    autocommit=True,
)

third_task >> fourth_task

fifth_task = PythonOperator(
    task_id='insert_authors',
    dag=dag,
    trigger_rule='none_failed',
    python_callable=insert_categories,
    op_kwargs={
        'output_folder': DATA_FOLDER,
    },
)

fourth_task >> fifth_task

sixth_task = PostgresOperator(
    task_id='insert_authors_to_db',
    dag=dag,
    postgres_conn_id='airflow_pg',
    sql='insert_authors.sql',
    trigger_rule='none_failed',
    autocommit=True,
)

fifth_task >> sixth_task

seventh_task = PythonOperator(
    task_id='insert_articles',
    dag=dag,
    trigger_rule='none_failed',
    python_callable=insert_articles,
    op_kwargs={
        'output_folder': DATA_FOLDER,
    },
)

sixth_task >> seventh_task

eightth_task = PostgresOperator(
    task_id='insert_articles_to_db',
    dag=dag,
    postgres_conn_id='airflow_pg',
    sql='insert_articles.sql',
    trigger_rule='none_failed',
    autocommit=True,
)

seventh_task >> eightth_task

ninth_task = PythonOperator(
    task_id='insert_article_author',
    dag=dag,
    trigger_rule='none_failed',
    python_callable=insert_articles_authors,
    op_kwargs={
        'output_folder': DATA_FOLDER,
    },
)

eightth_task >> ninth_task

tenth_task = PostgresOperator(
    task_id='insert_article_authors_to_db',
    dag=dag,
    postgres_conn_id='airflow_pg',
    sql='insert_article_author.sql',
    trigger_rule='none_failed',
    autocommit=True,
)

ninth_task >> tenth_task

